In [18]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transform
import timm
from transformers import Dinov2Config, Dinov2Model

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
accuracy_score(np.array([[0, 1], [1, 1]]), np.ones((2, 2)), normalize=False)

ModuleNotFoundError: No module named 'sklearn'

In [28]:
model = timm.create_model('vit_small_patch14_reg4_dinov2.lvd142m', pretrained=True)

# Change the classification head
num_classes = 10  # new number of classes
model.head = torch.nn.Sequential(torch.nn.Linear(model.num_features, num_classes), torch.nn.Sigmoid())


In [31]:
test = torch.rand(200,3,518,518)

model.eval()
with torch.inference_mode():
    out = model(test)
    
print(out.shape)

torch.Size([200, 10])


In [2]:
training_path = "data/Training_data/light_dataset"

In [4]:
training_transform = transform.Compose([transform.Resize((224, 224)), transform.ToTensor()])
train_dataset = torchvision.datasets.ImageFolder(root=training_path, transform=training_transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=12, shuffle=False)

In [5]:
def get_mean_std(loader):
    mean = 0.
    std = 0.
    total_img_cnt = 0
    for images, _ in loader:
        img_cnt_batch = images.size(0)
        images = images.view(img_cnt_batch, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_img_cnt += img_cnt_batch
        
    mean /= total_img_cnt
    std /= total_img_cnt
    
    return mean, std

In [6]:
get_mean_std(train_loader)

KeyboardInterrupt: 